# WorldClim annual precipitation sum and AOI masking

This notebook builds an annual precipitation raster by summing the 12 monthly WorldClim precipitation TIFFs, then masks that annual raster to each AOI shapefile (reprojecting AOIs as needed).


In [4]:
import os
import glob
from pathlib import Path
import math

import numpy as np
import rasterio
from rasterio.mask import mask
import geopandas as gpd


## Configure input/output paths

Set the folders for the monthly WorldClim rasters, the derived annual raster, the AOI shapefiles, and where masked outputs will be written.

In [ ]:
monthly_dir = r"D:\\Spatial Data\\Climate\\wc2.1_30s_prec"
annual_output = os.path.join(monthly_dir, "wc2.1_30s_prec_annual_sum.tif")
aoi_dir = r"C:\\Darwin geospatial\\25.01 OpenPAS\\AOI Shapefiles"
masked_output_dir = r"D:\\OpenPas Spatial Data\\Precipitation annual sum"

os.makedirs(masked_output_dir, exist_ok=True)
print(f"Monthly rasters: {monthly_dir}")
print(f"Annual output: {annual_output}")
print(f"AOI folder: {aoi_dir}")
print(f"Masked outputs: {masked_output_dir}")

## Build annual precipitation raster (sum)

Stream over the 12 monthly rasters block-by-block to sum them into an annual precipitation raster without loading everything into memory. Checks that all monthly rasters match in size, transform, and CRS.

In [ ]:
monthly_files = sorted(glob.glob(os.path.join(monthly_dir, "wc2.1_30s_prec_??.tif")))
if len(monthly_files) != 12:
    print(f"Warning: expected 12 monthly rasters, found {len(monthly_files)}")
    if not monthly_files:
        raise FileNotFoundError("No monthly rasters found. Check monthly_dir and file pattern.")

sources = [rasterio.open(tif) for tif in monthly_files]
try:
    template = sources[0]
    profile = template.profile
    nodata_value = profile.get("nodata")

    for src in sources[1:]:
        if (
            src.width != template.width
            or src.height != template.height
            or src.transform != template.transform
            or src.crs != template.crs
        ):
            raise ValueError("Monthly rasters differ in size/transform/CRS; check inputs.")

    output_nodata = nodata_value if nodata_value is not None else -9999.0
    profile.update({"dtype": "float32", "count": 1, "nodata": output_nodata})

    block_height, block_width = template.block_shapes[0]
    n_block_rows = math.ceil(template.height / block_height)
    n_block_cols = math.ceil(template.width / block_width)
    total_blocks = n_block_rows * n_block_cols
    progress_every = max(1, total_blocks // 20)  # about 5% steps
    print(f"Processing {total_blocks} blocks (block size: {block_width}x{block_height})")

    with rasterio.open(annual_output, "w", **profile) as dst:
        for idx, (_, window) in enumerate(template.block_windows(1), start=1):
            sum_arr = np.zeros((window.height, window.width), dtype="float64")
            count_arr = np.zeros((window.height, window.width), dtype="float64")
            for src in sources:
                block = src.read(1, window=window).astype("float64", copy=False)
                if nodata_value is not None:
                    valid = block != nodata_value
                    sum_arr[valid] += block[valid]
                    count_arr[valid] += 1
                else:
                    sum_arr += block
                    count_arr += 1

            sum_block = np.full(sum_arr.shape, output_nodata, dtype="float64")
            valid_pixels = count_arr > 0
            sum_block[valid_pixels] = sum_arr[valid_pixels]
            dst.write(sum_block.astype("float32"), 1, window=window)

            if idx % progress_every == 0 or idx == total_blocks:
                pct = (idx / total_blocks) * 100
                print(f"Processed {idx}/{total_blocks} blocks ({pct:.1f}% done)")
finally:
    for src in sources:
        src.close()

print(f"Annual precipitation sum written to {annual_output}")

## Mask annual raster to each AOI

Each AOI shapefile is reprojected to the raster CRS if needed, then used to crop/mask the annual raster. One masked TIFF is written per AOI.

In [ ]:
shapefiles = sorted(Path(aoi_dir).glob("*.shp"))
if not shapefiles:
    raise FileNotFoundError(f"No shapefiles found in {aoi_dir}")

total_aoi = len(shapefiles)
print(f"Processing {total_aoi} AOIs from {aoi_dir}")

with rasterio.open(annual_output) as src:
    annual_crs = src.crs
    annual_meta = src.meta.copy()

for i, shp in enumerate(shapefiles, start=1):
    gdf = gpd.read_file(shp)
    if gdf.empty:
        print(f"Skipping empty AOI: {shp}")
        continue

    if gdf.crs != annual_crs:
        gdf = gdf.to_crs(annual_crs)

    with rasterio.open(annual_output) as src:
        out_image, out_transform = mask(src, gdf.geometry, crop=True)

    out_meta = annual_meta.copy()
    out_meta.update({
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
    })

    out_path = os.path.join(masked_output_dir, f"{shp.stem}_precip_annual_sum.tif")
    with rasterio.open(out_path, "w", **out_meta) as dest:
        dest.write(out_image)

    print(f"[{i}/{total_aoi}] Saved masked raster: {out_path}")